# immoDB

## Presentation du Projet

Ce projet à pour but d'exploiter les données Open source de l'immobillier francais à travers diverses applications.

## Source de données
Notre source de données provient de Kaggle, site réputé, qui permet de récupérer et partager des sets de données mais aussi de pouvoir avoir une note de fiabilité et d'usabilité pour chaque dataset, le set *"immobilier france"* étant basé sur des sources officielles comme le *DVF+*, *IRCOM*, *la banque de france* et le *LOVAC* il obtient donc une note de *100% en crédibilité* et *100% en usabilité* grâce à un dataset documenté.


Nous pouvons donc récupérer notre première [source](https://www.kaggle.com/datasets/benoitfavier/immobilier-france?select=transactions.npz) en format ```.npz```, format utilisé pour stocker des **arrays numpy**. Ce fichier comprend l'ensemble des transactions immobilières depuis 2014, ce sera le fichier central dans notre projet. Pour pouvoir lire ces données et les charger dans un **dataframe pandas** on peut utiliser le snippet de code fourni avec le dataset.

In [12]:
import numpy as np
import pandas as pd

file = "../sources/transactions.npz"
arrays = dict(np.load(file))
data = {k: [s.decode("utf-8") for s in v.tobytes().split(b"\x00")] if v.dtype == np.uint8 else v for k, v in arrays.items()}
original_transactions = pd.DataFrame.from_dict(data)

## Exploration de données

Regardons la structure de notre dataset.

In [13]:
print(original_transactions.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8318280 entries, 0 to 8318279
Data columns (total 20 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   id_transaction              int32         
 1   date_transaction            datetime64[ns]
 2   prix                        float64       
 3   departement                 object        
 4   id_ville                    int32         
 5   ville                       object        
 6   code_postal                 int32         
 7   adresse                     object        
 8   type_batiment               object        
 9   vefa                        bool          
 10  n_pieces                    int32         
 11  surface_habitable           int32         
 12  id_parcelle_cadastre        object        
 13  latitude                    float64       
 14  longitude                   float64       
 15  surface_dependances         object        
 16  surface_locaux_ind

D'après la methode ```.info()``` nous avons un total de **8 318 280 lignes** et **20  colonnes** pour un total de **4.7 GB** de mémoire.

Cet quantité de données va nous permettre d'entrainer un modèle de *Machine Learning* de façon qualitative mais necessite un traitement approfondit afin de supprimer un maximum de valeurs abérantes et d'erreurs ainsi qu'un stockage partitioné permettant de gagner un maximum de temps et de performances sur nos requêtes.   

## Nettoyage des données

Malgré les sources qualitatives dont est composé notre set, quelques erreurs ou cas particuliers peuvent compromettre l'entrainement de notre futur modèle. Nous devons donc établir des règles impartiales afin de considérer notre futur modèle previsionnel comme représentant la réalité. 

Pour commencer nous allons verifier qu'il n'y ai pas de valeur null ou NaN dans notre dataset.

In [14]:
print(original_transactions.isna().sum())

id_transaction                0
date_transaction              0
prix                          0
departement                   0
id_ville                      0
ville                         0
code_postal                   0
adresse                       0
type_batiment                 0
vefa                          0
n_pieces                      0
surface_habitable             0
id_parcelle_cadastre          0
latitude                      0
longitude                     0
surface_dependances           0
surface_locaux_industriels    0
surface_terrains_agricoles    0
surface_terrains_sols         0
surface_terrains_nature       0
dtype: int64


Il n'y a donc aucune valeur null dans tout le dataset ✅


Il nous faut maintenant verifier le typage des colonnes numériques afin d'être sûr qu'une string ne soit pas dans une des entrées.

In [15]:
print(original_transactions[["prix", "surface_habitable", "n_pieces"]].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8318280 entries, 0 to 8318279
Data columns (total 3 columns):
 #   Column             Dtype  
---  ------             -----  
 0   prix               float64
 1   surface_habitable  int32  
 2   n_pieces           int32  
dtypes: float64(1), int32(2)
memory usage: 126.9 MB
None



Pour cela nous devons donc établir un seuil pour le prix au m² à partir duquel la donnée est considérée absurde comparée aux autres entrées semblables.\
Nous avons donc décidé d'exclure toutes les entrées dont le prix/m² est superieur à la moyenne plus 3 fois l'écart-type ou inferieur à la médiane moins 3 fois l'écart-type des entrées de la même année et du même département, c'est à dire aux extrêmes de la répartition statistique.
![schema ecart-type](../images/std.png)




In [16]:
import time, warnings
warnings.simplefilter("ignore")

def proc(df : pd.DataFrame):
    # calculate the average
    grouped_stats = df.groupby([pd.to_datetime(df['date_transaction']).dt.year, 'departement'])[
        ['prix', 'surface_habitable']].apply(
        lambda x: (x['prix'] / x['surface_habitable']).agg(['median', 'std'])).reset_index(drop=False)
    # create a year column to join `grouped_stats` and initial dataframe
    df['year'] = pd.to_datetime(df['date_transaction']).dt.year
    # merge `grouped_stats` and initial dataframe
    to_clean_df = pd.merge(df, grouped_stats, left_on=['year', 'departement'],
                           right_on=['date_transaction', 'departement'], suffixes=('', '_stats'))
    # remove absurd values where price/m² is 3 times more or less than the standard deviation
    filtered_df = to_clean_df[
        ((to_clean_df['prix'] / to_clean_df['surface_habitable']) < to_clean_df['median'] + 2 * to_clean_df['std']) & ((to_clean_df['prix'] / to_clean_df['surface_habitable']) > to_clean_df['median'] - 2 * to_clean_df['std'])]
    # drop temp. column
    filtered_df = filtered_df.drop(columns=['median', 'std', 'year'])
    return filtered_df

initial_len = len(original_transactions)

original_transactions = proc(original_transactions)
f"{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))} - [INIT]: {initial_len - len(original_transactions)} row suppressed ! That represent {((initial_len - len(original_transactions)) / len(original_transactions)) * 100:.2f}% of data"


'2024-06-24 16:57:24 - [INIT]: 202171 row suppressed ! That represent 2.49% of data'

## Stockage

mportation des Données dans MySQL Cloud : Explications Détaillées
L'importation des données dans MySQL Cloud se fait en plusieurs étapes. Voici une explication détaillée de chaque étape avec des exemples de code :

1. Configuration de la Connexion à la Base de Données
Pour connecter votre application à la base de données MySQL hébergée sur Google Cloud, nous utilisons SQLAlchemy. Assurez-vous d'avoir les informations de connexion stockées de manière sécurisée, par exemple, dans un fichier .env.

Fichier .env :